# Parameter estimation using nonlinear-monotone data

In this notebook we illustrate how to use nonlinear-monotone data for parameter optimization in pyPESTO.

We define a dataset of nonlinear-monotone data as $\{\widetilde{z}_k\}_{k=1}^N$ such that there exists a monotone (often unknown exactly) function $f$ which defines the relationship of the data and the model output:
$$\widetilde{z}_k = f(y_k(t_k, \theta), \theta) + \varepsilon_k, \quad k = 1, ..., N$$
Where:
- $\{y_k\}_{k=1}^N$ is the model output at timepoints $\{t_k\}_{k=1}^N$, 
- $\{\varepsilon_k\}_{k=1}^N$ is the measurement noise (usually normally distributed), 
- and $\theta$ is the vector of model (unknown) dynamical parameters.

This type of data can, for instance, be a result of Förster resonance energy transfer (FRET) measurements or saturated Western blots. 

In pyPESTO, we have implemented an alogorithm which constructs and optimizes a spline approximation $s(y, \xi)$ (piecewise linear function to be exact) of the nonlinear-monotone function $f(y_k(t_k, \theta), \theta)$. All model parameters are estimated hierarchically:
- The dynamical parameters $\theta$ are optimized in the outer hiearchical loop,
- The spline parameters $\xi$ are optimized in the inner loop for each iteration of the outer one.

In the following we will demonstrate how to use the spline approximation approach for integration of nonlinear-monotone data.

## Problem specification & importing model from the petab_problem

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import petab

import pypesto
import pypesto.logging
import pypesto.optimize as optimize
import pypesto.petab
from pypesto.C import LIN, InnerParameterType
from pypesto.hierarchical.spline_approximation import (
    SplineInnerProblem,
    SplineInnerSolver,
)
from pypesto.hierarchical.spline_approximation.parameter import (
    SplineInnerParameter,
)
from pypesto.hierarchical.spline_approximation.visualize import (
    plot_splines_from_inner_result,
    plot_splines_from_pypesto_result,
)

To specify usage of nonlinear-monotone data, only `nonlinear_monotone=True` has to be passed to the constructor of the `PetabImporter`:

In [ ]:
petab_folder = './example_nonlinear_monotone/'
yaml_file = 'example_nonlinear_monotone.yaml'

petab_problem = petab.Problem.from_yaml(petab_folder + yaml_file)

# To allow for optimization with nonlinear_monotone measurements,
# set nonlinear_monotone=True, when constructing the importer
importer = pypesto.petab.PetabImporter(petab_problem, nonlinear_monotone=True)

The petab_problem has to be specified in the usual PEtab formulation. The nonlinear-monotone measurements have to be specified in the `measurement.tsv` file by adding  `nonlinear_monotone` in the new `measurementType` column and determining the `measurementGroup`, where group specifies that the measurements come from the same nonlinear monotone function:

In [ ]:
from pandas import option_context

with option_context('display.max_colwidth', 400):
    display(petab_problem.measurement_df)

## Constructing the objective and pypesto problem

Different options can be used for the spline approximations:
- `spline_ratio`: float value, determines the number of spline knots as `n_spline_pars` = `spline_ratio` * `n_datapoints`  
- `use_minimal_difference` : `True` / `False`

The `use_minimal_difference` option determines whether a minimal difference between the spline heights will be constrained. For most models, enabling this option will increase convergence by increasing error of incorrect ordering. Nevertheless, the option has to be used with caution, as it will reduce the spline's capability of approximating functions with flat regions well. This we will show in this notebook.

Now when we construct the `objective`, it will construct all objects of the optimal scaling inner optimization:
- `SplineInnerSolver`
- `SplineAmiciCalculator`
- `SplineInnerProblem`

Specifically, the `SplineInnerSolver` and `SplineInnerProblem` will be constructed with default settings of 
- `spline_ratio` = 1/2
- `use_minimal_difference` = True

In [ ]:
objective = importer.create_objective()

To give non-default options to the `OptimalScalingInnerSolver` and `OptimalScalingProblem`, one can pass them as arguments when constructing the `objective`:

In [ ]:
objective = importer.create_objective(
    inner_solver_options={
        "spline_ratio": 1 / 2,
        "use_minimal_difference": True,
    },
)

Alternatively, one can even pass them to the importer constructor `pypesto.petab.PetabImporter()`.

If changing the `spline_ratio` setting, one has to create the objective object again, as this requires a constuction of the new `SplineInnerProblem` object with the requested amount of inner parameters.

Now let's construct the pyPESTO problem and optimizer. We're going to use a gradient-based optimizer for a faster optimization, but gradient-free optimizers can be used in the same way:

In [ ]:
problem = importer.create_problem(objective)

engine = pypesto.engine.SingleCoreEngine()

optimizer = optimize.ScipyOptimizer(
    method="L-BFGS-B",
    options={"disp": None, "ftol": 2.220446049250313e-09, "gtol": 1e-5},
)
n_starts = 10

## Running optimization using spline approximation

Now running optimization is as simple as running usual pyPESTO miminization:

In [ ]:
np.random.seed(n_starts)

result = optimize.minimize(
    problem, n_starts=n_starts, optimizer=optimizer, engine=engine
)

The model optimization has good convergence with a plateu at the optimal point:

In [ ]:
from pypesto.visualize import parameters, waterfall

waterfall([result], size=(10, 3))
plt.show()
parameters([result], size=(10, 3))
plt.show()

We can plot the optimized spline of the best start using the `plot_from_pypesto_result` visualization:

In [ ]:
plot_splines_from_pypesto_result(result)
plt.show()

## Caution when enabling minimal difference

To illustrate that minimal difference sometimes has a negative effect we will apply it to a very simple synthetic "model" -- simulation of the exponential function:

In [ ]:
timepoints = np.linspace(0, 10, 11)
function = np.exp

simulation = timepoints
sigma = np.full(len(timepoints), 1)

# Create synthetic data as the exponential function of timepoints
data = function(timepoints)

spline_ratio = 1 / 2
n_spline_pars = int(np.ceil(spline_ratio * len(timepoints)))


par_type = 'spline'
mask = [np.full(len(simulation), True)]

inner_parameters = [
    SplineInnerParameter(
        inner_parameter_id=f'{par_type}_{1}_{par_index+1}',
        inner_parameter_type=InnerParameterType.SPLINE,
        scale=LIN,
        lb=-np.inf,
        ub=np.inf,
        ixs=mask,
        index=par_index + 1,
        group=1,
    )
    for par_index in range(n_spline_pars)
]

inner_problem = SplineInnerProblem(
    xs=inner_parameters, data=[data], spline_ratio=spline_ratio
)

options = {
    'minimal_diff_on': {
        'spline_ratio': 1 / 2,
        'use_minimal_difference': True,
    },
    'minimal_diff_off': {
        'spline_ratio': 1 / 2,
        'use_minimal_difference': False,
    },
}
inner_solvers = {}
results = {}

for minimal_diff, option in options.items():
    inner_solvers[minimal_diff] = SplineInnerSolver(
        options=option,
    )

    # Solve the inner problem to obtain the optimal spline
    results[minimal_diff] = inner_solvers[minimal_diff].solve(
        problem=inner_problem,
        sim=[simulation],
        sigma=[sigma],
    )

    plot_splines_from_inner_result(inner_problem, results[minimal_diff])
    plt.show()

The optimized spline for the case with enabled minimal difference is performing much worse. This is due to the relative flatness of the data with respect to the true model output.

The minimal difference is determined as $$\text{min\_diff} = \frac{\text{measurement\_range}}{2\cdot \text{n\_inner\_pars}}$$ 
so for nonlinear-monotone functions which are relatively flat on some intervals, it is best to keep the minimal difference disabled.

As the true output (e.g. observable simulation of the model with true parameters) is mostly a-priori not known, it's hard to know whether the minimal difference is going to have a bad or good effect on the optimization. So a good heuristic is to run both and compare results.